In [1]:
from ipywidgets import interact
from brian2 import *
import NeuronEquations
import BrianHF
from ipywidgets import interact


@interact(tau=(1, 100), vt=(0, 30), vr=(0, 10), v0=(0, 30), gL=(1, 10),
          K_GABA=(0, 10), K_AMPA=(0, 10), taugd=(0.25, 10, 0.25), tauad=(0.25, 10, 0.25),
          taugr=(0.25, 10, 0.25), tauar=(0.25, 10, 0.25), w_N = (0, 100, 0.5), w_G = (0, 100, 0.5),
          w_A = (0, 100, 0.5))
def simulate_neurons(tau=10, vt=15, vr=0, v0=0, gL=1, K_GABA=1, K_AMPA=10, taugd=5,
                     tauad=2, taugr=0.25, tauar=0.5, w_N=80, w_G=30, w_A=50):
    start_scope()

    grid_size_x = 4 # Number of neurons in the x direction
    grid_size_y = 4
    N_Neurons = grid_size_x * grid_size_y # Number of neurons

    # Neuron Parameters
    Num_Neighbours = 2
    tau = tau*ms # Time constant
    vt = vt*mV # Threshold Voltage
    vr = vr*mV # Reset Voltage
    v0 = v0*mV # Resting Voltage - Not necessarily used
    gL = gL/ms # Leak Conductance - Not necessarily used

    # Neuron Equation
    Eqs_Neurons = NeuronEquations.EQ_LIF_N

    # Synapse Parameters
    K_GABA = K_GABA
    K_AMPA = K_AMPA
    taugd = taugd * ms
    tauad = tauad * ms
    taugr = taugr * ms
    tauar = tauar * ms
    
    

    # Input generation
    indices = np.concatenate((arange(N_Neurons), arange(5), arange(7,11)))
    times = np.concatenate(([1]*N_Neurons, [2]*5, [3]*4))*ms
    G_Spikes = SpikeGeneratorGroup(N_Neurons, indices, times)
    
    G_Neurons = NeuronGroup(N_Neurons, Eqs_Neurons, threshold='v>vt', reset='v = vr', refractory=0 * ms, method='euler')

    # Generate x and y values for each neuron
    x_values = np.repeat(np.arange(grid_size_x), grid_size_y)
    y_values = np.tile(np.arange(grid_size_y), grid_size_x)

    # Assign x, y, and z values to each neuron
    G_Neurons.X = x_values
    G_Neurons.Y = y_values
    
 # Creating the synapses
    Syn_Input_Neurons = Synapses(G_Spikes, G_Neurons, 'w : volt/second', on_pre='Ia += w')
    Syn_Neurons_GABA = Synapses(G_Neurons, G_Neurons, 'w : volt/second', on_pre='Ig -= w')
    Syn_Neurons_AMPA = Synapses(G_Neurons, G_Neurons, 'w : volt/second', on_pre='Ia += w')

    # SYNAPSE CONNECTIONS:
    # Connect the first synapses from input to G_neurons on a 1 to 1 basis
    Syn_Input_Neurons.connect(condition= 'i==j')
    # Connect the second group of synapses from a neuron to its neighbors
    Syn_Neurons_GABA.connect(condition='i != j and abs(X_pre - X_post) <= Num_Neighbours and abs(Y_pre - Y_post) <= Num_Neighbours')
    # Connect the last set of synapses from a neuron to itself (recurrent)
    Syn_Neurons_AMPA.connect(condition='i == j')

    # Define the weights of the synapses
    Syn_Input_Neurons.w = w_N * volt/second
    Syn_Neurons_GABA.w = w_G * volt/second
    Syn_Neurons_AMPA.w = w_A * volt/second
    
    # Monitor the spikes
    SpikeMon_Input = SpikeMonitor(G_Spikes)
    SpikeMon_Neurons = SpikeMonitor(G_Neurons)

    # Monitor the state variables
    StateMon_Neurons = StateMonitor(G_Neurons, ['Ia', 'Ig', 'I', 'v'], record=True)
    
    
    # Run the simulation
    run(100*ms)
    
    # Plot the results
    figure()
    subplot(211)
    plot(SpikeMon_Input.t/ms, SpikeMon_Input.i, '.k')
    xlabel('Time (ms)')
    ylabel('Neuron index')

    # Plot the results
    subplot(212)
    plot(SpikeMon_Neurons.t/ms, SpikeMon_Neurons.i, '.k')
    xlabel('Time (ms)')
    ylabel('Neuron index')
    show()
    
   #  BrianHF.visualise_neurons_states(StateMon_Neurons, [4, 5], 'all')

WARNING    /home/iit.local/bmaacaron/code/ERO-SNN/BrianHF.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
 [py.warnings]


interactive(children=(IntSlider(value=10, description='tau', min=1), IntSlider(value=15, description='vt', max…